In [1]:
import openpyxl as xl
import requests 
from bs4 import BeautifulSoup

In [2]:
class Player:    
    def __init__(self, name, score, link, img):
        self.name = name
        self.score = score
        self.link = link
        self.img = img
        self.position = ""
        self.height = 0
        self.weight = 0
        self.dob = ""
        self.team = ""
        
    def setDetails(self, position, positionNo, height, weight, dob, team):
        self.position = position
        self.positionNo = positionNo
        self.height = height
        self.weight = weight
        self.dob = dob
        self.team = team
    
    def getName(self):
        return self.name

    def getScore(self):
        return self.score

    def getLink(self):
        return self.link

    def getImg(self):
        return self.img

    def getPosition(self):
        return self.position
    
    def getPositionNo(self):
        return self.positionNo

    def getHeight(self):
        return self.height

    def getWeight(self):
        return self.weight

    def getDob(self):
        return self.dob

    def getTeam(self):
        return self.team

In [3]:
class Date:    
    def __init__(self, day, month, year):
        self.day = day        
        self.month = month                
        self.year = year

    def getDay(self):
        return self.day

    def getMonth(self):
        return self.month

    def getYear(self):
        return self.year

In [4]:
def scrapePlayers():
    
    playerList = []

    page = requests.get('https://index.rugbypass.com/rpi/six-nations/all/all/7-days/high-to-low/players/')
    content = BeautifulSoup(page.content, 'html.parser')
    players = content.find_all(class_='player-bar')
    
    for p in players:
        newPlayer = Player(p.find(class_='name').string, p.find(class_='display-score').string, p['href'], p.find('img')['src'])
        playerList.append(newPlayer)
    return playerList

In [1]:
#to check scrape player function

# for x in scrapePlayers(): 
#     print(x.getName() + "\t--> " + x.getScore())
#     print("\tlink\t--> " + x.getLink())
#     print("\timg\t--> " + x.getImg())

In [2]:
def scrapeDetails(player):
    page = requests.get(player.link)
    content = BeautifulSoup(page.content, 'html.parser')
    
    position = "-"
    
    positionNo = 0

    height = 0

    weight = 0

    day = 0

    month = 0

    year = 0

    team = "-"
    
    positions = content.find_all(class_='positions')
    
    for p in positions:
        position = p.get_text('div')[1:-1]
        positionList = ["Loose Head Prop", 
                        "Hooker", 
                        "Tight Head Prop", 
                        "Lock 4", 
                        "Lock 5", 
                        "Blindside Flanker", 
                        "Openside Flanker", 
                        "Number 8", 
                        "Scrum Half", 
                        "Fly Half", 
                        "Left Wing", 
                        "Inside Centre", 
                        "Outside Centre", 
                        "Right Wing", 
                        "Fullback"]
        positionNo = positionList.index(position) + 1

    measurements = content.find_all(class_='measurement')

    for m in measurements:
        if ('divHeightdiv' in m.get_text('div')):
            heightStr = m.get_text('div').replace("divHeightdiv", "")
            if (heightStr.replace(" ", "") != "---"):
                height = float(heightStr.replace(" ", "")[:-1])

        elif ('divWeightdiv' in m.get_text('div')):
            weightStr = m.get_text('div').replace("divWeightdiv", "")
            if (weightStr.replace(" ", "") != "---"):
                weight = int(weightStr.replace(" ", "")[:-2])


        elif ('divDOBdiv' in m.get_text('div')):
            date =  m.get_text('div').replace("divDOBdiv", "")
            date = date.replace(" ", "")
            if (date != "---"):
                day = int(date[0:2])
                monthStr = date[2:5]

                months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                for mon in months:
                    if mon in monthStr:
                        month = months.index(monthStr) + 1
                year = int(date[5:9])

    teams = content.find_all(class_='player-team')

    for t in teams:
        if ("England" in t.get_text()):
            team = "England"

        elif ("France" in t.get_text()):
            team = "France"

        elif ("Ireland" in t.get_text()):
            team = "Ireland"

        elif ("Italy" in t.get_text()):
            team = "Italy"

        elif ("Scotland" in t.get_text()):
            team = "Scotland"

        elif ("Wales" in t.get_text()):
            team = "Wales"

    dob = Date(day, month, year)
    player.setDetails(position, positionNo, height, weight, dob, team)

In [7]:
#  month proofing as a int

#         months = [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec]
#         for mon in months:
#              if mon in month: